In [1]:
import xarray as xr
import numpy as np

In [2]:
# Load WBT and temp data
wbt = xr.open_mfdataset('../data/processed/CESM2/WBTdailymax/*',combine='nested',concat_dim='ensemble',chunks={'time':1095})
t_ref = xr.open_mfdataset('../data/processed/CESM2/TSAdailymax/*',combine='nested',concat_dim='ensemble',chunks={'time':1095})

In [3]:
# Calculate WBGT
wbgt = 0.7*wbt['WBT'] + 0.3*(t_ref['TSA']-273.15)

# Adjust for upper and lower bound of WBGT-capacity interpolation
wbgt = wbgt.where(wbgt>25,25)
wbgt = wbgt.where(wbgt<33,33)

# Convert from WBGT to capacity
cap_daily = 100 - 25*((wbgt-25)**(2/3))

# Go from daily capacity to annual capacity
cap_yearly = cap_daily.resample(time='1Y').mean()

In [4]:
cap_yearly

,Array,Chunk
Bytes,134.92 MB,221.18 kB
Shape,"(5, 122, 192, 288)","(1, 1, 192, 288)"
Count,6750 Tasks,610 Chunks
Type,float32,numpy.ndarray
